# Notebook to preprocess the LFM2019 dataset
Please run the preceeding notebook, "filter_lfm2019.ipynb" before running this notebook.

In [2]:
import os
import csv
import json
import glob
import utils
import pickle 
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from scipy import sparse as sp
import matplotlib.pyplot as plt
from collections import defaultdict

In [3]:
data_dir = "/media/gustavo/Storage/Datasets/PersonalityBias/pers-bias-paper/"
# data_dir = r"F:\Temp\lfm2b data"

In [4]:
!ls -l /media/gustavo/Storage/Datasets/PersonalityBias/pers-bias-paper

total 6612
-rwxrwxrwx 1 root root 4062706 Aug  6  2020 inter.csv
-rwxrwxrwx 1 root root 1863655 Aug  6  2020 pers.csv
-rwxrwxrwx 1 root root  841503 Aug  6  2020 track_ids.csv


### User data

In [6]:
# Load user data
df_users = pd.read_csv(os.path.join(data_dir, "pers.csv"), index_col=None)
df_users.rename(columns={'user_id':'userID'},inplace=True)
df_users

,userID,agr,con,ext,neu,ope
0,14601,0.029778,0.577021,0.079205,0.030381,0.409037
1,13400,0.064731,0.044838,0.183774,0.935121,0.408691
2,5057,0.726240,0.902584,0.958060,0.082035,0.908693
3,9112,0.104627,0.389257,0.200606,0.534548,0.896833
4,14388,0.567508,0.075580,0.132370,0.655471,0.318356
...,...,...,...,...,...,...
18305,13220,0.335081,0.193584,0.482927,0.376366,0.965331
18306,10756,0.370308,0.139981,0.158871,0.410347,0.811481
18307,14411,0.823859,0.573187,0.396910,0.630984,0.317753
18308,13069,0.309801,0.019558,0.173226,0.791738,0.659325


In [9]:
df_items = pd.read_csv(os.path.join(data_dir, "track_ids.csv"))
df_items.columns = ["itemID"] +list(df_items.columns[1:])
df_items

,itemID,track_artist,artist,track
0,0,DNA.;Kendrick Lamar,Kendrick Lamar,DNA.
1,1,Coffee;BTS,BTS,Coffee
2,2,Love You to Death;Type O Negative,Type O Negative,Love You to Death
3,3,Mask Off;Future,Future,Mask Off
4,4,Redbone;Childish Gambino,Childish Gambino,Redbone
...,...,...,...,...
15748,15748,Evermore;Paris & Simo,Paris & Simo,Evermore
15749,15749,Pleaser;Wallows,Wallows,Pleaser
15750,15750,Seize the Block;Migos,Migos,Seize the Block
15751,15751,Glow Up;Mary J. Blige,Mary J. Blige,Glow Up


### Interaction matrix

As there are by any means too much items, we will randomly select some items and put them in a sparse matrix.

In [14]:
import pickle 
from pathlib import Path
path_data_dir = Path(data_dir)
interactions=  pd.read_csv(path_data_dir/"inter.csv", names=["userID","itemID"] )
#user_features = pickle.load(open(path_data_dir/ "","rb") )


TypeError: read_csv() got an unexpected keyword argument 'columns'

In [15]:
interactions

,user_id,track_id
0,0,0
1,22,0
2,47,0
3,64,0
4,64,0
...,...,...
395051,16184,15480
395052,16414,15480
395053,16528,15480
395054,16529,15480


In [13]:

unique_items = interactions["track_id"].unique()
print(f"n_items: {len(unique_items)}")
item2token = pd.Series(unique_items)
token2item = pd.Series(data=item2token.index, index=item2token.values)


n_items: 15753


In [4]:


iids_dict = {k:token2item.loc[value].values  for k,value in interaction_dict.items()}
uids_iids_array = pd.Series(iids_dict).explode().reset_index().to_numpy().astype(np.int32)
n_users,n_items = len(interactions()),len(unique_items) 
data = np.ones(uids_iids_array.shape[0],dtype=np.int8)
uids,iids = uids_iids_array[:,0],uids_iids_array[:,1]
interaction_matrix = sp.csr_matrix((data, (uids, iids)), 
                                       (n_users, n_items))

n_items: 1544646


In [12]:
sp.save_npz(os.path.join(data_dir, "interaction_matrix_personality.npz"),interaction_matrix)

In [8]:
n_sampled_items = 100_000
np.random.seed(42)

interaction_matrix = sp.load_npz(os.path.join(data_dir, "interaction_matrix_personality.npz"))
n_users, n_items = interaction_matrix.shape

sampled_items = sorted(np.random.choice(np.arange(n_items), n_sampled_items, replace=False))

interaction_matrix_sampled = interaction_matrix[:, sampled_items]
interaction_matrix_sampled

<1470x100000 sparse matrix of type '<class 'numpy.int8'>'
	with 410280 stored elements in Compressed Sparse Row format>

### User information

In [ ]:
# Moreover, as the didn't seem to be any input validation in place when collecting the data,
# we will set the users' attributes to default if they 
#    - don't make sense (also in case of additional punctations, etc..),
#    - belong to a group with only a few users (for nominal data)

min_n_users_per_group = 200
bad_location_groups = [k for k, v in df_users["Country"].value_counts().items() if v < min_n_users_per_group]
bad_location_groups += ["", ",", np.nan]

bad_location_users = df_users["Country"].isin(bad_location_groups)
print(f"{bad_location_users.sum()} of {len(bad_location_users)} user countries reset to default")

df_users.loc[bad_location_users, "Country"] = ""
df_users

In [ ]:
min_age = 10
max_age = 100
bad_age_users = (df_users["Age"] < min_age) | (max_age < df_users["Age"])

print(f"{bad_age_users.sum()} of {len(bad_age_users)} user ages reset to default")
df_users.loc[bad_age_users, "Age"] = np.NaN
df_users

In [ ]:
# We group users in age categories to make the data easier to handle
age_categories = [18, 25, 35, 45, 55]

assigned_age_cat = np.zeros(shape=(len(df_users),) , dtype=int)
for cat in age_categories:
    assigned_age_cat += (df_users["Age"] > cat).astype(int)

assigned_age_cat[df_users["Age"].isna()] = -1
df_users["Age"] = assigned_age_cat
df_users["Age"].value_counts()

In [ ]:
# Extracted the README file
attribute_descriptions = {
    "gender": {
        "m": "male",
        "f": "female"
    },
    "age": {
        -1: "unknown",
        0:  "Under 18",
        1:  "18-24",
        2:  "25-34",
        3:  "35-44",
        4:  "45-54",
        5:  "55+"
    },
    # just in case scripts require for each attribute to have a description
    "country": {d:d for d in df_users["Country"].unique()}
}
attribute_descriptions["country"][""] = "undefined"

In [9]:
n_interactions = interaction_matrix_sampled.sum()
density = n_interactions / (n_items * n_users)

# Show some statistics about the dataset
print("Number of users:", interaction_matrix_sampled.shape[0])
print("Number of items:", interaction_matrix_sampled.shape[1])

print("\nNumber of interactions:", n_interactions)
print(f"Density: {density:.6f}")

Number of users: 1470
Number of items: 100000

Number of interactions: 2023227
Density: 0.000891


### Data preparation
For our use-case our end-result should be binary interaction matrix, where ```1``` denotes that a user
interacted with an item, and ```0``` that they did not.

In [10]:
# some items might be missing, let's therefore adjust / re-enumerate the indices
item_ids = df_items["itemID"].unique()
item_rename_dict = token2item.to_dict()
item_rename_dict_reverse = item2token.to_dict()

df_items = df_items[df_items["itemID"].isin(item_ids)]

#df_items["itemID"]= token2item.loc[df_items["itemID"].values ]
#print(token2item.loc[df_items["itemID"].values]              )

df_items["itemID_int"] = token2item.loc[df_items["itemID"]].values
df_items.rename(columns={"itemID":"itemID_str"},inplace=True)
df_items.rename(columns={"itemID_int":"itemID"},inplace=True)
item_ids = list(range(len(sampled_items)))
df_item_info = df_items[df_items["itemID"].isin(sampled_items)]

In [11]:
df_item_info

,itemID_str,spotify_popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,itemID
15,7zzbPkvxJ6iANIxcSz60rm,20.0,0.864000,0.638,161560.0,0.156,0.000000,8.0,0.1830,-17.372,1.0,0.0741,103.905,4.0,0.4430,1179763
20,7zzUVdPXpDr0o7Y2SzgWO8,25.0,0.007410,0.720,282000.0,0.494,0.836000,1.0,0.0786,-9.957,1.0,0.1590,176.955,4.0,0.9020,404914
35,7zzD8MpgOd9qd3i3fspKlf,8.0,0.037800,0.497,298878.0,0.580,0.000000,9.0,0.0943,-8.671,1.0,0.0287,147.024,4.0,0.3520,81011
37,7zzAU4EIuTPTUjiMUpmLbN,26.0,0.022500,0.572,158640.0,0.994,0.915000,5.0,0.1090,-5.855,0.0,0.0418,119.535,4.0,0.1270,350401
49,7zylvC67zTFvKLxSecOInv,20.0,0.002640,0.192,231360.0,0.936,0.054000,6.0,0.3080,-4.484,0.0,0.0749,167.153,4.0,0.4590,474268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544594,0012gcMTLOeEr2UfflyGgh,9.0,0.133000,0.613,210733.0,0.487,0.811000,0.0,0.0711,-9.202,0.0,0.0603,76.519,4.0,0.1080,287007
1544603,000rdOe90OF65lgFIwPGHm,0.0,0.324000,0.481,203333.0,0.552,0.000399,9.0,0.1170,-6.874,1.0,0.0338,145.629,4.0,0.2460,1278576
1544611,000cAxMxUfuDlvPYIVrrZ4,28.0,0.000147,0.474,181345.0,0.966,0.000000,8.0,0.3740,-2.957,1.0,0.0931,156.993,4.0,0.6020,75032
1544616,000V2mp9CgAS1Er4eTzG3C,5.0,0.845000,0.151,286960.0,0.238,0.829000,0.0,0.0922,-13.381,1.0,0.0389,105.631,3.0,0.0286,742785


In [12]:
# create new user file for our usage
df_user_info = df_users.copy()
interaction_matrix = interaction_matrix_sampled

# change column names to camel-case & drop hyphens
rn = {cn: cn[0].lower() + cn[1:].replace("-", "") for cn in df_user_info.columns}
df_user_info.rename(rn, inplace=True, axis=1)

sampled_suffix = f"_{n_sampled_items}"

In [13]:
# store results
attribute_descriptions = {}
storage_dir = os.path.join(data_dir, f"full" + sampled_suffix)
utils.store_results(storage_dir, interaction_matrix, df_user_info, attribute_descriptions)

In [15]:
# filter users & tracks with too less interaction
min_interactions_user = 5
min_interactions_item = 5

im_all, umap_all, imap_all, uinfo_all, iinfo_all = utils.ensure_min_interactions(interaction_matrix, 
                                                      min_interactions_user, min_interactions_item,
                                                      df_user_info, df_item_info)

utils.print_stats(im_all)

# account for previous adjustment of item indices
#imap_all = imap_all.assign(old=imap_all["old"].replace(item_rename_dict_reverse))
imap_all["old_new"] = item2token.loc[imap_all["old"]].values
imap_all.rename(columns={"old":"old_prev"},inplace=True)
imap_all.rename(columns={"old_new":"old"},inplace=True)
assert im_all.shape[0] == len(uinfo_all)

# store results
storage_dir = os.path.join(data_dir, f"user_gte_{min_interactions_user}_item_gte_{min_interactions_item}" + sampled_suffix)
utils.store_results(storage_dir, im_all, uinfo_all, attribute_descriptions, iinfo_all, umap_all, imap_all)

Final shape of interactions matrix is (1284, 37929)
==> 1284 users and 37929 items are remaining.

Number of interactions is 1936313,
which leads to a density of 0.0398.


In [ ]:
# drop all data where no gender info is given
mask_gender_given = df_user_info["gender"].isin(["f", "m"])
user_info_gen = df_user_info[mask_gender_given]
im_gen = interaction_matrix[mask_gender_given, :]

im_gen, umap_gen, imap_gen, uinfo_gen, iinfo_gen = utils.ensure_min_interactions(im_gen, 
                                                      min_interactions_user, min_interactions_item,
                                                      user_info_gen, df_item_info)

utils.print_stats(im_gen)

# account for previous adjustment of item indices
imap_gen = imap_gen.assign(old=imap_gen["old"].replace(item_rename_dict_reverse))

assert im_gen.shape[0] == len(uinfo_gen)

# store results
storage_dir = os.path.join(data_dir, f"user_gte_{min_interactions_user}_item_gte_{min_interactions_item}_gender" + sampled_suffix)
utils.store_results(storage_dir, im_gen, uinfo_gen, attribute_descriptions, iinfo_gen, umap_gen, imap_gen)

In [ ]:
# drop all data where no location info is given
mask_country_given = df_user_info["country"] != ""
user_info_country = df_user_info[mask_country_given]
im_country = interaction_matrix[mask_country_given, :]

im_country, umap_country, imap_country, uinfo_country, iinfo_country = utils.ensure_min_interactions(im_country, 
                                                      min_interactions_user, min_interactions_item,
                                                      user_info_country, df_item_info)

utils.print_stats(im_country)

# account for previous adjustment of item indices
item_rename_dict_reverse = {v: k for k, v in item_rename_dict.items()}
imap_country = imap_country.assign(old=imap_country["old"].replace(item_rename_dict_reverse))

assert im_country.shape[0] == len(uinfo_country)

# store results
storage_dir = os.path.join(data_dir, f"user_gte_{min_interactions_user}_item_gte_{min_interactions_item}_loc" + sampled_suffix)
utils.store_results(storage_dir, im_country, uinfo_country, attribute_descriptions, iinfo_country, umap_country, imap_country)

In [ ]:
# drop all data where no age info is given
mask_age_given = df_user_info["age"] != -1
user_info_age = df_user_info[mask_age_given]
im_age = interaction_matrix[mask_age_given, :]

im_age, umap_age, imap_age, uinfo_age, iinfo_age = utils.ensure_min_interactions(im_age, 
                                                      min_interactions_user, min_interactions_item,
                                                      user_info_age, df_item_info)

utils.print_stats(im_age)

# account for previous adjustment of item indices
item_rename_dict_reverse = {v: k for k, v in item_rename_dict.items()}
imap_age = imap_age.assign(old=imap_age["old"].replace(item_rename_dict_reverse))

assert im_age.shape[0] == len(uinfo_age)

# store results
storage_dir = os.path.join(data_dir, f"user_gte_{min_interactions_user}_item_gte_{min_interactions_item}_age" + sampled_suffix)
utils.store_results(storage_dir, im_age, uinfo_age, attribute_descriptions, iinfo_age, umap_age, imap_age)

In [ ]:
# drop all data where either age or loc is not given
mask_given = mask_gender_given
mask_given &= mask_country_given
mask_given &= mask_age_given

user_info = df_user_info[mask_given]
im = interaction_matrix[mask_given, :]

im, umap, imap, uinfo, iinfo = utils.ensure_min_interactions(im, 
                                                      min_interactions_user, min_interactions_item,
                                                      user_info, df_item_info)

utils.print_stats(im)

# account for previous adjustment of item indices
item_rename_dict_reverse = {v: k for k, v in item_rename_dict.items()}
imap = imap.assign(old=imap["old"].replace(item_rename_dict_reverse))

assert im.shape[0] == len(uinfo)

# store results
storage_dir = os.path.join(data_dir, f"user_gte_{min_interactions_user}_item_gte_{min_interactions_item}_gender_age_loc" + sampled_suffix)
utils.store_results(storage_dir, im, uinfo, attribute_descriptions, iinfo, umap, imap)

In [18]:
# drop all data where either age or loc is not given
mask_given = mask_gender_given
mask_given &= mask_country_given
mask_given &= mask_age_given

user_info = df_user_info[mask_given]
im = interaction_matrix[mask_given, :]

im, umap, imap, uinfo, iinfo = utils.ensure_min_interactions(im, 
                                                      min_interactions_user, min_interactions_item,
                                                      user_info, df_item_info)

utils.print_stats(im)

# account for previous adjustment of item indices
item_rename_dict_reverse = {v: k for k, v in item_rename_dict.items()}
imap = imap.assign(old=imap["old"].replace(item_rename_dict_reverse))

assert im.shape[0] == len(uinfo)

# store results
storage_dir = os.path.join(data_dir, f"user_gte_{min_interactions_user}_item_gte_{min_interactions_item}_gender_age_loc" + sampled_suffix)
utils.store_results(storage_dir, im, uinfo, attribute_descriptions, iinfo, umap, imap)

Final shape of interactions matrix is (7603, 62617)
==> 7603 users and 62617 items are remaining.

Number of interactions is 1845963,
which leads to a density of 0.0039.
